# FBSDE

Ji, Shaolin, Shige Peng, Ying Peng, and Xichuan Zhang. “Three Algorithms for Solving High-Dimensional Fully-Coupled FBSDEs through Deep Learning.” ArXiv:1907.05327 [Cs, Math], February 2, 2020. http://arxiv.org/abs/1907.05327.

Consider an FBSDE:

$$
\begin{aligned}
dX_t^i &= \exp(-X_t^i) \sum_j Z^{ij}_t dW^i_t & X_0 = 1\\
dY_t^i &= \sum_j Z^{ij}_t dW^i_t + \frac 12 \exp(-X^i_t) \sum_j (Z^{ij}_t)^2 dt & Y_T = \exp(X^i_T)
\end{aligned}
$$

One can see by Ito's lemma that

$$
Y^i_t = \exp(X^i_t)
$$

In [1]:
import os
from makers.gpu_utils import *
os.environ["CUDA_VISIBLE_DEVICES"] = str(pick_gpu_lowest_memory())

In [4]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Lambda, Reshape, concatenate, Layer, BatchNormalization
from keras import Model, initializers
from keras.callbacks import ModelCheckpoint
from keras.metrics import mean_squared_error
import matplotlib.pyplot as plt
from datetime import datetime
from keras.metrics import mse
from keras.optimizers import Adam
from keras.callbacks import Callback

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num GPUs Available:  1


# Inputs

In [109]:
# numerical parameters
n_paths = 2 ** 16
n_timesteps = 16
T = 0.1

In [111]:
def b(t, x, y, z, r):
    return [0., 0.]

def s(t, x, y, z, r):
    return [
        [tf.exp(-x[0]) * z[0][0], tf.exp(-x[0]) * z[0][1]],
        [tf.exp(-x[1]) * z[1][0], tf.exp(-x[1]) * z[1][1]]
    ]

def f(t, x, y, z, r):
    return [
        tf.exp(-x[0]) * (z[0][0]**2 + z[0][1]**2) / 2.,
        tf.exp(-x[1]) * (z[1][0]**2 + z[1][1]**2) / 2.,
    ]

def v(t, x, y, z, r):
    return [[0., 0.], [0., 0.]]

def g(x):
    return [tf.exp(x[0]), tf.exp(x[1])]

# Automatic parameters

In [112]:
n_dimensions = 2
n_diffusion_factors = 2
n_jump_factors = 2
dt = T / n_timesteps

# Initial value layer

In [113]:
class InitialValue(Layer):
    
    def __init__(self, y0, **kwargs):
        super().__init__(**kwargs)
        self.y0 = y0
    
    def call(self, inputs):
        return self.y0


# Model

In [114]:
def dX(t, x, y, z, r, dW, dN):
    
    def drift(arg):
        x, y, z, r = arg
        return tf.math.multiply(b(t, x, y, z, r), dt)
    a0 = tf.vectorized_map(drift, (x, y, z, r))
        
    def noise(arg):
        x, y, z, r, dW = arg
        return tf.tensordot(s(t, x, y, z, r), dW, [[1], [0]])
    a1 = tf.vectorized_map(noise, (x, y, z, r, dW))

    def jump(arg):
        x, y, z, r, dN = arg
        return tf.tensordot(v(t, x, y, z, r), dN, [[1], [0]])
    a2 = tf.vectorized_map(jump, (x, y, z, r, dN))
        
    return a0 + a1 + a2

def dY(t, x, y, z, r, dW, dN):
    
    def drift(arg):
        x, y, z, r = arg
        return tf.math.multiply(f(t, x, y, z, r), dt)
    a0 = tf.vectorized_map(drift, (x, y, z, r))

    def noise(arg):
        x, y, z, r, dW = arg
        return tf.tensordot(z, dW, [[1], [0]])
    a1 = tf.vectorized_map(noise, (x, y, z, r, dW))
    
    def jump(arg):
        x, y, z, r, dN = arg
        return tf.tensordot(r, dN, [[1], [0]])
    a2 = tf.vectorized_map(jump, (x, y, z, r, dN))
    
    return a0 + a1 + a2

In [115]:
paths = []

n_hidden_units = n_dimensions + n_diffusion_factors + n_jump_factors + 10

inputs_dW = Input(shape=(n_timesteps, n_diffusion_factors))
inputs_dN = Input(shape=(n_timesteps, n_jump_factors))

x0 = tf.Variable([[1., 2.]], trainable=False)
y0 = tf.Variable([[0., 0.]], trainable=True)

x = InitialValue(x0, trainable=False, name='x_0')(inputs_dW)
y = InitialValue(y0, trainable=True, name='y_0')(inputs_dW)

z = concatenate([x, y])
z = Dense(n_hidden_units, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name='z1_0')(z)
z = Dense(n_dimensions * n_diffusion_factors, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name='z2_0')(z)
z = BatchNormalization(name='zbn_0')(z)
z = Reshape((n_dimensions, n_diffusion_factors), name='zr_0')(z)

r = concatenate([x, y])
r = Dense(n_hidden_units, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name='r1_0')(r)
r = Dense(n_dimensions * n_jump_factors, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name='r2_0')(r)
r = BatchNormalization(name='rbn_0')(r)
r = Reshape((n_dimensions, n_jump_factors), name='rr_0')(r)

paths += [[x, y, z, r]]

# pre-compile lambda layers

@tf.function
def hx(args):
    i, x, y, z, r, dW, dN = args
    return x + dX(i * dt, x, y, z, r, dW, dN)

@tf.function
def hy(args):
    i, x, y, z, r, dW, dN = args
    return y + dY(i * dt, x, y, z, r, dW, dN)

for i in range(n_timesteps):
    
    step = InitialValue(tf.Variable(i, dtype=tf.float32, trainable=False))(inputs_dW)
    
    dW = Lambda(lambda x: x[0][:, tf.cast(x[1], tf.int32)])([inputs_dW, step])
    dN = Lambda(lambda x: x[0][:, tf.cast(x[1], tf.int32)])([inputs_dN, step])
    
    x, y = (
        Lambda(hx, name=f'x_{i+1}')([step, x, y, z, r, dW, dN]),
        Lambda(hy, name=f'y_{i+1}')([step, x, y, z, r, dW, dN]),
    )
        
    # we don't train z for the last time step; keep for consistency
    z = concatenate([x, y])
    z = Dense(n_hidden_units, activation='relu', name=f'z1_{i+1}')(z)
    z = Dense(n_dimensions * n_diffusion_factors, activation='relu', name=f'z2_{i+1}')(z)
    z = Reshape((n_dimensions, n_diffusion_factors), name=f'zr_{i+1}')(z)
    z = BatchNormalization(name=f'zbn_{i+1}')(z)
    
    # we don't train r for the last time step; keep for consistency
    r = concatenate([x, y])
    r = Dense(n_hidden_units, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name=f'r1_{i+1}')(r)
    r = Dense(n_dimensions * n_jump_factors, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=1e-1), name=f'r2_{i+1}')(r)
    r = Reshape((n_dimensions, n_jump_factors), name=f'rr_{i+1}')(r)
    r = BatchNormalization(name=f'rbn_{i+1}')(r)

    paths += [[x, y, z, r]]
    
outputs_loss = Lambda(lambda r: r[1] - tf.transpose(tf.vectorized_map(g, r[0])))([x, y])
outputs_paths = tf.stack(
    [tf.stack([p[0] for p in paths[1:]], axis=1), tf.stack([p[1] for p in paths[1:]], axis=1)] + 
    [tf.stack([p[2][:, :, i] for p in paths[1:]], axis=1) for i in range(n_diffusion_factors)] +
    [tf.stack([p[3][:, :, i] for p in paths[1:]], axis=1) for i in range(n_jump_factors)], axis=2)


model_loss = Model([inputs_dW, inputs_dN], outputs_loss)

# (n_sample, n_timestep, x/y/z_k, n_dimension)
# skips the first time step
model_paths = Model([inputs_dW, inputs_dN], outputs_paths)


In [ ]:
model_loss.summary()

# Transfer weights (if needed)

In [ ]:
# try transfer learning from another starting point

model_loss.get_layer('y_0').set_weights(m_large.get_layer('y_0').get_weights())

for i in range(n_timesteps):
    model_loss.get_layer(f'z1_{i}').set_weights(m_large.get_layer(f'z1_{i}').get_weights())
    model_loss.get_layer(f'z2_{i}').set_weights(m_large.get_layer(f'z2_{i}').get_weights())

In [ ]:
# transfer learning from cruder discretization

model_loss.get_layer('y_0').set_weights(m_small.get_layer('y_0').get_weights())

n_small = 4

for i in range(n_small):
    for j in range(n_timesteps // n_small):
        model_loss.get_layer(f'z1_{n_timesteps // n_small * i}').set_weights(m_small.get_layer(f'z1_{i}').get_weights())
        model_loss.get_layer(f'z2_{n_timesteps // n_small * i}').set_weights(m_small.get_layer(f'z2_{i}').get_weights())
        model_loss.get_layer(f'z1_{n_timesteps // n_small * i + j}').set_weights(m_small.get_layer(f'z1_{i}').get_weights())
        model_loss.get_layer(f'z2_{n_timesteps // n_small * i + j}').set_weights(m_small.get_layer(f'z2_{i}').get_weights())

# Training

In [117]:
dW = tf.sqrt(dt) * tf.random.normal((n_paths, n_timesteps, n_diffusion_factors))
dN = tf.random.poisson((n_paths, n_timesteps), tf.constant(dt * np.array([1., 1.]).transpose().reshape(-1)))
target = tf.zeros((n_paths, n_dimensions))

In [56]:
# check for exploding gradients before training

with tf.GradientTape() as tape:
    loss = mse(model_loss([dW, dN]), target)

# bias of the last dense layer
variables = model_loss.variables[-1]
tape.gradient(loss, variables)

In [118]:
# tensorboard log dir
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
model_name = f"{timestamp}__{n_dimensions}__{n_paths}__{n_timesteps}__{T}"
log_dir = "/home/tmp/starokon/tensorboard/" + model_name

# callbacks
# checkpoint_callback = ModelCheckpoint('_models/weights{epoch:04d}.h5', save_weights_only=True, overwrite=True)
checkpoint_callback = ModelCheckpoint(f'_output/{model_name}/model.h5', save_weights_only=True, save_best_only=True, overwrite=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=1)
nan_callback = tf.keras.callbacks.TerminateOnNaN()

In [119]:
np.set_printoptions(edgeitems=11, linewidth=90, formatter=dict(float=lambda x: "%7.5g" % x))

In [120]:
adam = Adam(learning_rate=1e-3)
model_loss.compile(loss='mse', optimizer=adam)

In [121]:
class Y0Callback(Callback):
    
    def __init__(self):
        super(Y0Callback, self).__init__()
        self.y0s = []
    
    def on_epoch_end(self, epoch, logs=None):
        y0 = model_loss.variables[1].numpy()[0]
        self.y0s += [y0]
        print(y0 + "\n")
        
    def on_train_end(self, logs=None):
        pass

y0_callback = Y0Callback()

In [ ]:
history = model_loss.fit([dW, dN], target, batch_size=128, initial_epoch=0, epochs=100, callbacks=[nan_callback, checkpoint_callback, tensorboard_callback, y0_callback])


In [69]:
# validate

dW_test = tf.sqrt(dt) * tf.random.normal((n_paths//8, n_timesteps, n_diffusion_factors))
dN_test = tf.random.poisson((n_paths//8, n_timesteps), tf.constant(dt * np.array([1., 1.]).transpose().reshape(-1)))
target_test = tf.zeros((n_paths//8, n_dimensions))

model_loss.evaluate([dW_test, dN_test], target_test)

256/256 [==============================] - 4s 2ms/step - loss: 3.4178e-09


3.417778726699794e-09

# Display paths and loss

In [ ]:
# load bad model
model_loss.load_weights('_models/weights0011.h5')

In [55]:
loss = model_loss([dW, dN]).numpy()
loss

array([[-4.21302719e-03, -1.83777374e-04,  7.68974721e-02, ...,
        -1.08535380e-04, -3.65010686e-02, -1.43885612e-04],
       [ 3.12728644e-03, -1.86214020e-04,  1.65139437e-02, ...,
        -1.07569365e-04,  9.47822630e-02, -1.43885612e-04],
       [-1.13846944e-03, -1.84155477e-04,  3.47495675e-02, ...,
        -1.07487824e-04, -5.74785471e-03, -1.43885612e-04],
       ...,
       [-4.19508014e-03, -1.82968608e-04,  9.62089002e-03, ...,
        -1.07971777e-04, -5.63402846e-03, -1.43885612e-04],
       [-1.20397564e-02, -1.82203512e-04,  1.12469167e-01, ...,
        -1.08954977e-04, -2.62902267e-02, -1.43885612e-04],
       [-3.71439802e-03, -1.83611395e-04,  4.11325842e-02, ...,
        -1.07773740e-04,  1.11734495e-02, -1.43885612e-04]], dtype=float32)

In [59]:
paths = model_paths([dW, dN]).numpy()